In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import os

2021-09-11 07:21:20.887125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-11 07:21:20.887145: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Before running cells below,be sure to remove the below files

### These files contain less samples or no samples at all. Therefore these files are not usable.

Bad Files
 - OpenBCI-RAW-2021-08-19_15-09-22.txt(Right)
 - OpenBCI-RAW-2021-08-19_15-02-47.txt(Right)
 - OpenBCI-RAW-2021-08-19_15-03-12.txt(Right)
 - OpenBCI-RAW-2021-08-19_15-03-03.txt(Right)
 - OpenBCI-RAW-2021-08-19_15-00-14.txt(Left)
 - OpenBCI-RAW-2021-08-19_15-00-07.txt(Left)
 - OpenBCI-RAW-2021-08-19_15-00-25.txt(Left)
 - OpenBCI-RAW-2021-08-19_15-00-42.txt(Left)
 - OpenBCI-RAW-2021-08-19_14-59-40.txt(left)
 - OpenBCI-RAW-2021-08-19_15-00-54.txt(left)
 
 
 
 


In [2]:
def prepare_pandas_dataset():
    dataset_root_folder = "thoughts"
    dataset = pd.DataFrame()
    class_folders = os.listdir(dataset_root_folder)
    timeframe_start = 251
    timeframe_end = 1501
    for class_folder in class_folders:
        if class_folder == ".ipynb_checkpoints" or class_folder == "pick_object":
            continue
        for file in os.listdir(dataset_root_folder+"/"+class_folder):
            file_name = dataset_root_folder+"/"+class_folder+"/"+file 
            if file == ".ipynb_checkpoints":
                continue
            if dataset.empty:
                dataset = pd.read_csv(file_name,skiprows=6,header=None)
                dataset=dataset.iloc[timeframe_start:timeframe_end,1:9]
                dataset["class"]=class_folder
            else:
                tmp_dataset=pd.read_csv(file_name,skiprows=6,header=None)
                tmp_dataset=tmp_dataset.iloc[timeframe_start:timeframe_end,1:9]
                tmp_dataset["class"]=1 if class_folder == "left" else 2 
                dataset=dataset.append(tmp_dataset,ignore_index=True)
            
    return dataset
        

In [3]:
# this function returns a tuple containing signals and labels (signals,labels)

def prepare_tuple_dataset():
    dataset_root_folder = "thoughts"
    signals = []
    labels = []
    class_folders = os.listdir(dataset_root_folder) #folders of seperate signals like left,right and pick_object
    #timeframe_starrt and timeframe_end selects from which sample to which sample
    # should be used for training and evaluation 
    timeframe_start = 251 #from which sample to st
    timeframe_end = 1501
    for class_folder in class_folders:
        if class_folder == ".ipynb_checkpoints" or class_folder == "pick_object":
            continue
        for file in os.listdir(dataset_root_folder+"/"+class_folder):
            file_name = dataset_root_folder+"/"+class_folder+"/"+file 
            if file == ".ipynb_checkpoints":
                continue
            else:
                tmp_dataset = pd.read_csv(file_name,skiprows=6,header=None)
                tmp_dataset=tmp_dataset.iloc[timeframe_start:timeframe_end,1:9]
                signals.append(tmp_dataset.T.to_numpy().reshape(8,1250))
                labels.append(np.array(1 if class_folder == "left" else 0).reshape(1,1))
            
    return signals,labels

from sklearn.preprocessing import MinMaxScaler
def scale_signals(signals):
    for i in range(len(signals)):
        for j in range(len(signals[i])):
            scaler = MinMaxScaler()
            signals[i] = [scaler.fit_transform(channel.reshape(1,-1)).ravel() for channel in signals[i]]
            
    return signals
        

In [4]:
from scipy.signal import stft
import librosa
def calculate_stft(signals,frame_size=250,hop_length=125,sampling_frequency=250.0):
    stfts = []
    for signal in signals:
        channel_stfts=[]
        for channel in signal:
            channel_stft=librosa.stft(channel,
                                    n_fft=frame_size,
                                    hop_length=hop_length
                                    )
            channel_stfts.append(np.abs(channel_stft)**2)
        stfts.append(np.array(channel_stfts))
    return stfts

In [5]:
signals,labels = prepare_tuple_dataset()
signals = calculate_stft(signals)


ValueError: cannot reshape array of size 896 into shape (8,1250)

In [ ]:
for i in range(len(signals)):
    signals[i]=signals[i].reshape((126,11,8))
    print("shape",signals[i].shape)
    

In [ ]:
signals[0].shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Flatten,Dense,InputLayer,Reshape,Bidirectional,Conv2D,MaxPool2D,BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.optimizers import Adam,SGD
def build_model():
    model = Sequential([
        Conv2D(128,(2,2),padding="same",input_shape=(126,11,8),activation="leaky_relu",kernel_initializer="he_normal"),
        MaxPool2D((2,2)),
        Conv2D(256,(2,2),activation="leaky_relu",kernel_initializer="he_normal",padding="same"),
        Conv2D(512,(2,2),activation="leaky_relu",kernel_initializer="he_normal",padding="same"),        
        MaxPool2D((2,2)),
        Flatten(),
        Dense(1,activation="softmax",name="output_layer"),
    ])
    model.compile(loss="binary_crossentropy",optimizer=Adam(0.000001),metrics=["accuracy"])
    return model
model = build_model()
model.summary()

In [ ]:
model = build_model()
history = model.fit(tf_dataset,epochs=epochs,validation_data=tf_dataset)

loss_train = history.history['loss']
# loss_val = history.history['val_loss']

epochs_range = range(1,epochs+1)
plt.plot(epochs_range, loss_train, 'g', label='Training loss')
# plt.plot(epochs_range, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(tf_dataset)
predictions

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
def plot_spectrogram(signal,sampling_rate,hop_length,y_axis="linear"):
    
  plt.figure(figsize=(25,10))
  librosa.display.specshow(signal,
                            sr=sampling_rate,
                            hop_length=hop_length,
                            x_axis="time",
                            y_axis=y_axis)
  plt.colorbar(format="%+2.f")


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Flatten,Dense,InputLayer,Reshape,Bidirectional
from tensorflow.keras import Sequential
def build_model():
    model = Sequential([
        InputLayer((8,1250)),
        Reshape((125,80)),
        LSTM(units=64,name="LSTM_1"),
        Dense(32,activation="relu"),
        Dense(1,activation="sigmoid",name="output_layer")
    ])
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model
model = build_model()
model.summary()
model.fit(tf_dataset,epochs=epochs)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Flatten,Dense,InputLayer,Reshape,Bidirectional
from tensorflow.keras import Sequential
def build_model():
    model = Sequential([
        InputLayer((8,1250)),
        Reshape((125,80)),
        LSTM(units=64,name="LSTM_1"),
        Reshape((64,1)),
        LSTM(units=128,name="LSTM_2"),
        Dense(32,activation="relu"),
        Dense(1,activation="sigmoid",name="output_layer")
    ])
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model


In [ ]:
model.predict(tf_dataset)

In [ ]:
model.fit(tf_dataset,epochs=30)

In [ ]:
labels